In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np

In [2]:
artsjobs = pd.read_csv("../arts_jobs/artsjobs.csv",encoding='utf-8')
guardian = pd.read_csv("../guardian/guardian.csv",encoding='utf-8')
indeed = pd.read_csv("../indeed/indeed.csv",encoding='utf-8')
mbs = pd.read_csv("../music_business_worldwide/music_business_worldwide.csv",encoding='utf-8')
uk_mj = pd.read_csv("../music-jobs/uk_mus_jobs.csv",encoding='utf-8')
us_mj = pd.read_csv("../music-jobs/us_mus_jobs.csv",encoding='utf-8')
simply =  pd.read_csv("../simply_hired/simply_hired.csv",encoding='utf-8')
reed =  pd.read_csv("../reed/all_music_jobs.csv",encoding='utf-8')
glassdoor = pd.read_csv("../glassdoor/glassdoor.csv",encoding='utf-8')
mandy = pd.read_csv("../mandy/mandy.csv",encoding='utf-8')

In [3]:
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]

def dataset_info(dataset):
    var_name = namestr(dataset, globals())
    print("*********************")
    print("dataset: " + str(var_name))
    print("shape " + str(dataset.shape))
    print("head")
    print(dataset.head())
    print("description")
    print(dataset.describe())
    print("columns " + str(dataset.columns))
    print(dataset.isnull().sum())
    print("is dataframe? " + str(isinstance(dataset, pd.DataFrame)))
    print("*********************")

#https://stackoverflow.com/questions/592746/how-can-you-print-a-variable-name-in-python

In [4]:
dataset_info(artsjobs)
dataset_info(guardian)
dataset_info(indeed)
dataset_info(mbs)
dataset_info(uk_mj)
dataset_info(us_mj)
dataset_info(simply)
dataset_info(reed)
dataset_info(glassdoor)
dataset_info(mandy)

*********************
dataset: ['artsjobs']
shape (83, 4)
head
   Unnamed: 0                                       titleCompany    location  \
0           0       Stage Manager: Live & Digital, Clod Ensemble      London   
1           1  Trainee Producer for Creative Crawley micro-fe...      London   
2           2          Development Director, Plymouth Music Zone  South West   
3           3         Samba Tutor, SW19, Merton Music Foundation      London   
4           4  Digital Content Project Manager, Union Chapel ...      London   

                                      jobDescription  
0  Clod Ensemble is a multi-award winning, intern...  
1  Do you live in Crawley?\r\nAre you interested ...  
2  Plymouth Music Zone is looking for a dynamic a...  
3  Samba Tutor Vacancy\r\nNovember 2020 Start\r\n...  
4  To oversee the content development and deliver...  
description
       Unnamed: 0
count   83.000000
mean    10.036145
std      6.309972
min      0.000000
25%      5.000000
50%   

In [5]:
def simple_clean(dataset):
    new_index = len(dataset.index)
    numeric_values = list(range(0, new_index))
    dataset['new_index'] = numeric_values
    dataset = dataset.set_index('new_index')
    dataset = dataset.drop_duplicates()
    dataset = dataset.applymap(lambda s:s.lower() if type(s) == str else s)
    dataset = dataset.applymap(lambda s:s.replace('[^a-zA-Z \n\.]'," ") if type(s) == str else s)

    if 'Unnamed: 0' in dataset:
        dataset = dataset.drop(columns=['Unnamed: 0'])
        
    return dataset
    
# Source
# https://stackoverflow.com/questions/39512002/convert-whole-dataframe-from-lower-case-to-upper-case-with-pandas
# https://www.kaggle.com/kandij/job-analysis-content-recommendation

In [6]:
clean_artsjobs = simple_clean(artsjobs)
print(clean_artsjobs)

                                                titleCompany  \
new_index                                                      
0               stage manager: live & digital, clod ensemble   
1          trainee producer for creative crawley micro-fe...   
2                  development director, plymouth music zone   
3                 samba tutor, sw19, merton music foundation   
4          digital content project manager, union chapel ...   
5          consultant: development of cultural strategy f...   
6                        events officer, oxford city council   
7                                    trainee 2021, unlimited   
8          emergent commission opportunity for disabled a...   
9          events programmer (freelance), awards for youn...   
10         call out for blind and d/deaf artists and musi...   
11         marketing, media and digital content coordinat...   
12                           musinc 1:1 music tutors, musinc   
13         assistant music leader - newt

In [7]:
clean_artsjobs['title'] = clean_artsjobs.titleCompany.str.extract('(.*),[^,]*$')
clean_artsjobs['company'] = clean_artsjobs.apply(lambda row : row['titleCompany'].replace(str(row['title']), ''), axis=1)
clean_artsjobs['company'] = clean_artsjobs['company'].str.replace(",", "")
clean_artsjobs = clean_artsjobs.drop(columns=['titleCompany'])

#https://stackoverflow.com/questions/54892624/remove-substring-from-column-based-on-another-column

In [8]:
print(clean_artsjobs.company)

new_index
0                                       clod ensemble
1                                    creative crawley
2                                 plymouth music zone
3                             merton music foundation
4                                union chapel project
5                                   coast scarborough
6                                 oxford city council
7                                           unlimited
8                                         drake music
9                          awards for young musicians
10                                     lucy c hayward
11                                             musinc
12                                             musinc
13                                             musinc
14                                             musinc
15                                             musinc
16                                             musinc
17                                             musinc
18                

In [9]:
clean_guardian = simple_clean(guardian)
print(clean_guardian)

                                                  jobTitle  \
new_index                                                    
0                                       head of membership   
1                                theatre technical manager   
2                                          general manager   
3                                        credit controller   
4                                  specialist practitioner   
5                                       lead practitioners   
6                                                 producer   
7                                       associate teachers   
8                                         teacher of music   
9          music instrumental teacher for vocals (singing)   
10                          legacy and stewardship officer   
11                 supporter engagement and events officer   
12                       development manager (individuals)   
13                                        head of woodwind   
14      

In [10]:
clean_mbs = simple_clean(mbs)
print(clean_mbs)

                                                    jobTitle  \
new_index                                                      
0           pollen – head of hip-hop + r&b partnerships (us)   
1          universal music publishing group – junior prod...   
2           universal music publishing group – producer (uk)   
3          awal – global commercial partnership manager (...   
4          300 entertainment – digital marketing + stream...   
5               instrumental – head of business affairs (uk)   
6          warner music group – systems & asset administr...   
7                          bravado – marketing director (us)   
8          apple – head of international marketing, beats...   
9                      warner music group – vp, finance (us)   
10         anonymous – music management assistant / pa fo...   
11         disney music group – sr manager, marketing, ar...   
12         music gatewayâ – content marketing executive...   
13                        sostereo – mar

In [11]:
clean_mbs['company'] = clean_mbs['jobTitle'].str.split('–', 1).str[0]

clean_mbs['jobTitle'] = clean_mbs.apply(lambda row : row['jobTitle'].replace(str(row['company']), ''), axis=1)
clean_mbs['jobTitle'] = clean_mbs['jobTitle'].str.replace("–", "")

clean_mbs.fillna('***', inplace=True)
clean_mbs['location'] = clean_mbs.apply(lambda row : row['location'].replace(str(row['company']), ''), axis=1)

#https://stackoverflow.com/questions/54892624/remove-substring-from-column-based-on-another-column

In [12]:
clean_mandy = simple_clean(mandy)
print(clean_mandy)

                                                    jobTitle  \
new_index                                                      
0                             visiting music teacher - piano   
1                        junior intercompany royalty analyst   
2                                                   producer   
3                          development manager (individuals)   
4                                          lecturer in music   
5                                           teacher of music   
6                    music administrator and concert manager   
7                             visiting music teacher - flute   
8               lead practitioners – voice / technical sound   
9                                          head of marketing   
10                     administrator & projects co-ordinator   
11                 learning and development business partner   
12                                               piano tutor   
13                                      

In [13]:
clean_indeed = simple_clean(indeed)
print(clean_indeed)

                                                    jobTitle  \
new_index                                                      
0               freelance music journalists and broadcasters   
1          beautiful digital - digital account manager fo...   
2          music sales & business development representative   
3                                          guitar technician   
4                   unqualified primary teacher - asap start   
5                                    musical theatre teacher   
6          music required for ppa cover in a acton primar...   
7                                              head of music   
8              head music tutor / coordinator / music tutors   
9                                         freelance musician   
10         staff tutor (aligned to english, creative writ...   
11                       bookkeeper - part time (music shop)   
12                   music reviewer - albums and live events   
13                                     p

In [14]:
clean_glassdoor = simple_clean(glassdoor)
print(clean_glassdoor)

                                                    jobTitle  \
new_index                                                      
0          primary music teacher - immediate start (qts e...   
1                                              music teacher   
2                                             support worker   
3                              support worker/care assistant   
4                              finance assistant (part-time)   
5                 music editor, apple music – uk and ireland   
6                 software engineer (functional programming)   
7                                        content coordinator   
8                          music content strategist, youtube   
9                                        engineering manager   
10                         music content strategist, youtube   
11                        content manager (production music)   
12                                          studio assistant   
13                                      

In [15]:
clean_simply = simple_clean(simply)
print(clean_simply)

                                                    jobTitle  \
new_index                                                      
0                    music reviewer - albums and live events   
1                                                   musician   
2          sales executive – dj & music production equipment   
3                 piano/keyboard & vocal teacher - liverpool   
4                                           music journalist   
5                               cityrep(live music promoter)   
6                                      senior producer music   
7                        bookkeeper - part time (music shop)   
8                          music content strategist, youtube   
9                                    assistant music leaders   
10                                           music publicist   
11                            product sales manager ( music)   
12                           dance, music & singing teachers   
13         amazon music uk pr manager (1

In [16]:
clean_reed = simple_clean(reed)
print(clean_reed)

           applications  employerId                           employerName  \
new_index                                                                    
0                     3      432060                             academics    
1                    50      544143                     harmonic group ltd   
2                     8      427940                  tradewind recruitment   
3                    18      443541           education network birmingham   
4                     2      363367                     teaching personnel   
5                    12      116473  the music market   arts and media ltd   
6                     3      432060                             academics    
7                     6      427940                  tradewind recruitment   
8                     4      524744    hays specialist recruitment limited   
9                    10      116473  the music market   arts and media ltd   
10                   20      404559                            a

In [17]:
cols_to_drop = ['jobId', 'applications', 'employerId']
clean_reed = clean_reed.drop(columns=cols_to_drop)
print(clean_reed)

                                    employerName  \
new_index                                          
0                                     academics    
1                             harmonic group ltd   
2                          tradewind recruitment   
3                   education network birmingham   
4                             teaching personnel   
5          the music market   arts and media ltd   
6                                     academics    
7                          tradewind recruitment   
8            hays specialist recruitment limited   
9          the music market   arts and media ltd   
10                                   aj chambers   
11         the music market   arts and media ltd   
12                               career teachers   
13                                    academics    
14                            term time teachers   
15                            harmonic group ltd   
16                              reeson education   
17          

In [18]:
clean_uk_mj = simple_clean(uk_mj)
print(clean_uk_mj)

                                                    jobTitle  \
new_index                                                      
0                                 peripatetic violin teacher   
1          audio director [unannounced project] (365) (au...   
2          audio director [unannounced project] (365) (au...   
3                        trainee producer for micro-festival   
4                                   music leader (assistant)   
5                                        music teacher (ks2)   
6                                               music editor   
7                                          lead music leader   
8                                     assistant music leader   
9                                          lead music leader   
10                                                  producer   
11                                      development director   
12          marketing, media and digital content coordinator   
13         emergent commission opportuni

In [19]:
clean_us_mj = simple_clean(us_mj)
print(clean_us_mj)

                                                    jobTitle  \
new_index                                                      
0                                        live event producer   
1          assistant professor of electronic/electroacous...   
2          global commercial partnership manager (hip hop...   
3                                        editor, global hits   
4                                     program manager, music   
5                                    hip hop and r&b manager   
6                                    contract sound designer   
7                                           audio technician   
8                                             audio director   
9                    member relations and operations manager   
10                                manager, midwest promotion   
11                                        operations manager   
12                                radio production assistant   
13                                 senio

In [20]:
print(namestr(clean_artsjobs, globals()))
print(clean_artsjobs.columns)
print("********")
print(namestr(clean_guardian, globals()))
print(clean_guardian.columns)
print("********")
print(namestr(clean_indeed, globals()))
print(clean_indeed.columns)
print("********")
print(namestr(clean_mbs, globals()))
print(clean_mbs.columns)
print("********")
print(namestr(clean_uk_mj, globals()))
print(clean_uk_mj.columns)
print("********")
print(namestr(clean_us_mj, globals()))
print(clean_us_mj.columns)
print("********")
print(namestr(clean_simply, globals()))
print(clean_simply.columns)
print("********")
print(namestr(clean_reed, globals()))
print(clean_reed.columns)
print("********")
print(namestr(clean_glassdoor, globals()))
print(clean_glassdoor.columns)
print("********")
print(namestr(clean_mandy, globals()))
print(clean_mandy.columns)
print("********")

['clean_artsjobs']
Index(['location', 'jobDescription', 'title', 'company'], dtype='object')
********
['clean_guardian']
Index(['jobTitle', 'location', 'jobDescription', 'company'], dtype='object')
********
['clean_indeed']
Index(['jobTitle', 'jobDescription', 'location', 'companyName'], dtype='object')
********
['clean_mbs']
Index(['jobTitle', 'jobDescription', 'location', 'company'], dtype='object')
********
['clean_uk_mj']
Index(['jobTitle', 'location'], dtype='object')
********
['clean_us_mj']
Index(['jobTitle', 'location'], dtype='object')
********
['clean_simply']
Index(['jobTitle', 'location', 'jobDescription', 'company'], dtype='object')
********
['clean_reed']
Index(['employerName', 'jobDescription', 'jobTitle', 'locationName'], dtype='object')
********
['clean_glassdoor']
Index(['jobTitle', 'location', 'jobDescription', 'company'], dtype='object')
********
['clean_mandy']
Index(['jobTitle', 'location', 'jobType'], dtype='object')
********


In [21]:
clean_artsjobs = clean_artsjobs.rename(columns={"title": "job_title", "jobDescription": "job_description"})
print(clean_artsjobs.columns)

Index(['location', 'job_description', 'job_title', 'company'], dtype='object')


In [22]:
clean_mbs = clean_mbs.rename(columns={"jobTitle": "job_title", "jobDescription": "job_description"})
print(clean_mbs.columns)

Index(['job_title', 'job_description', 'location', 'company'], dtype='object')


In [23]:
clean_guardian = clean_guardian.rename(columns={"jobTitle": "job_title", "jobDescription": "job_description"})
print(clean_guardian.columns)

Index(['job_title', 'location', 'job_description', 'company'], dtype='object')


In [24]:
clean_indeed = clean_indeed.rename(columns={"companyName": "company","jobTitle": "job_title", "jobDescription": "job_description"})
print(clean_indeed.columns)

Index(['job_title', 'job_description', 'location', 'company'], dtype='object')


In [25]:
clean_uk_mj = clean_uk_mj.rename(columns={"jobTitle": "job_title"})
clean_uk_mj["company"] = "***"
clean_uk_mj["job_description"] = "***"
print(clean_uk_mj.columns)

Index(['job_title', 'location', 'company', 'job_description'], dtype='object')


In [26]:
clean_us_mj = clean_us_mj.rename(columns={"jobTitle": "job_title"})
clean_us_mj["company"] = "***"
clean_us_mj["job_description"] = "***"
print(clean_us_mj.columns)

Index(['job_title', 'location', 'company', 'job_description'], dtype='object')


In [27]:
clean_simply = clean_simply.rename(columns={"jobTitle": "job_title", "jobDescription": "job_description"})
print(clean_simply.columns)

Index(['job_title', 'location', 'job_description', 'company'], dtype='object')


In [28]:
clean_reed = clean_reed.rename(columns={"jobTitle": "job_title", "jobDescription": "job_description", "employerName": "company", "locationName": "location"})
print(clean_reed.columns)

Index(['company', 'job_description', 'job_title', 'location'], dtype='object')


In [29]:
clean_mandy = clean_mandy.rename(columns={"jobTitle": "job_title", "jobType": "job_description"})
clean_mandy["company"] = "***"
print(clean_mandy.columns)

Index(['job_title', 'location', 'job_description', 'company'], dtype='object')


In [30]:
clean_glassdoor = clean_glassdoor.rename(columns={"jobTitle": "job_title", "jobDescription": "job_description"})
print(clean_glassdoor.columns)

Index(['job_title', 'location', 'job_description', 'company'], dtype='object')


In [31]:
all_music_jobs = pd.DataFrame()
all_music_jobs = all_music_jobs.append( [clean_glassdoor, clean_mbs, clean_artsjobs, clean_mandy, clean_reed, clean_simply, clean_us_mj, clean_uk_mj, clean_indeed, clean_guardian], ignore_index=True, verify_integrity=True, sort=False)

In [32]:
all_music_jobs.head(200)

,job_title,location,job_description,company
0,primary music teacher - immediate start (qts e...,"london, england",NaN,.css-10l5u4p span{color:#505863;font-size:15px...
1,music teacher,"brentwood, england",NaN,engage education
2,support worker,"feltham, england",NaN,consensus
3,support worker/care assistant,"new barnet, england",NaN,norwood
4,finance assistant (part-time),"camden town, england",NaN,octave group
5,"music editor, apple music – uk and ireland","london, england",NaN,apple
6,software engineer (functional programming),"london, england",NaN,funding circle uk
7,content coordinator,"london, england",NaN,hogarth worldwide
8,"music content strategist, youtube","london, england",NaN,youtube
9,engineering manager,"london, england",NaN,funding circle uk


In [33]:
all_music_jobs.shape

(6507, 4)

In [34]:
all_music_jobs['job_description'].isnull().sum()

902

In [35]:
all_music_jobs['location'].isnull().sum()

1

In [36]:
all_music_jobs['company'].isnull().sum()

1

In [37]:
all_music_jobs.fillna("***", inplace=True)

In [38]:
all_music_jobs['job_description'].isnull().sum()

0

In [39]:
from uuid import uuid4

all_music_jobs['job_uuid'] = [uuid4() for _ in range(len(all_music_jobs.index))]

#https://stackoverflow.com/questions/48837006/add-uuid-to-a-new-column-in-a-pandas-dataframe

In [40]:
from nltk.corpus import stopwords

In [41]:
from nltk.stem import PorterStemmer
stemmer =  PorterStemmer()
stop = stopwords.words('english')

dataframe_columns = ['job_description', 'location', 'job_title']

In [42]:
def process_df_col():
    for col in dataframe_columns:
        column = all_music_jobs[col]
        columns = column.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
        column = column.apply(lambda x : filter(None,x.split(" ")))
        #column = column.apply(lambda x : [stemmer.stem(y) for y in x])
        column = column.apply(lambda x : " ".join(x))

#https://www.kaggle.com/kandij/job-analysis-content-recommendation

In [43]:
process_df_col()

In [44]:
all_music_jobs[~all_music_jobs.job_title.str.contains('support worker')]
all_music_jobs[~all_music_jobs.job_title.str.contains('nurse')]
all_music_jobs[~all_music_jobs.job_title.str.contains('science')]
all_music_jobs[~all_music_jobs.job_title.str.contains('nanny')]
all_music_jobs[~all_music_jobs.job_title.str.contains('work from home')]
all_music_jobs[~all_music_jobs.job_title.str.contains('vehicle')]

all_music_jobs[~all_music_jobs.company.str.contains('maria mallaband care group')]
all_music_jobs[~all_music_jobs.company.str.contains('nhs')]
all_music_jobs[~all_music_jobs.company.str.contains('paid surveys')]
#https://stackoverflow.com/questions/51182182/pandas-delete-row-if-cell-contains-specific-text

,job_title,location,job_description,company,job_uuid
0,primary music teacher - immediate start (qts e...,"london, england",***,.css-10l5u4p span{color:#505863;font-size:15px...,be2558c8-d135-44a6-a634-58252388346b
1,music teacher,"brentwood, england",***,engage education,207cbe4d-cef2-458f-a297-de474b906ef2
2,support worker,"feltham, england",***,consensus,6adcc88b-af33-4954-9aed-3927a275e489
3,support worker/care assistant,"new barnet, england",***,norwood,d6249591-047d-49a8-a635-d9cb31994453
4,finance assistant (part-time),"camden town, england",***,octave group,c6034db9-b995-4170-b10b-ad7653cbd588
5,"music editor, apple music – uk and ireland","london, england",***,apple,9ba138e8-b281-40b2-8a9e-eb2ecbd7f548
6,software engineer (functional programming),"london, england",***,funding circle uk,657ca789-fe31-4556-b158-0992502f2d7f
7,content coordinator,"london, england",***,hogarth worldwide,2ec30505-f43b-4a8b-91ad-faaaf55494fb
8,"music content strategist, youtube","london, england",***,youtube,b01bc7bf-7a47-4380-bf96-a013f51fddfb
9,engineering manager,"london, england",***,funding circle uk,5beea5e0-c2ab-4c5f-8c77-0a141a1c3bcc


In [45]:
all_music_jobs.shape

(6507, 5)

In [46]:
all_music_jobs["all_info"] = all_music_jobs["job_title"].map(str)+ " " + all_music_jobs["company"] +" "+ all_music_jobs["location"] + " " + all_music_jobs["job_description"]

In [47]:
all_music_jobs.shape

(6507, 6)

In [48]:
all_music_jobs.head(50)

,job_title,location,job_description,company,job_uuid,all_info
0,primary music teacher - immediate start (qts e...,"london, england",***,.css-10l5u4p span{color:#505863;font-size:15px...,be2558c8-d135-44a6-a634-58252388346b,primary music teacher - immediate start (qts e...
1,music teacher,"brentwood, england",***,engage education,207cbe4d-cef2-458f-a297-de474b906ef2,"music teacher engage education brentwood, engl..."
2,support worker,"feltham, england",***,consensus,6adcc88b-af33-4954-9aed-3927a275e489,"support worker consensus feltham, england ***"
3,support worker/care assistant,"new barnet, england",***,norwood,d6249591-047d-49a8-a635-d9cb31994453,support worker/care assistant norwood new barn...
4,finance assistant (part-time),"camden town, england",***,octave group,c6034db9-b995-4170-b10b-ad7653cbd588,finance assistant (part-time) octave group cam...
5,"music editor, apple music – uk and ireland","london, england",***,apple,9ba138e8-b281-40b2-8a9e-eb2ecbd7f548,"music editor, apple music – uk and ireland app..."
6,software engineer (functional programming),"london, england",***,funding circle uk,657ca789-fe31-4556-b158-0992502f2d7f,software engineer (functional programming) fun...
7,content coordinator,"london, england",***,hogarth worldwide,2ec30505-f43b-4a8b-91ad-faaaf55494fb,"content coordinator hogarth worldwide london, ..."
8,"music content strategist, youtube","london, england",***,youtube,b01bc7bf-7a47-4380-bf96-a013f51fddfb,"music content strategist, youtube youtube lond..."
9,engineering manager,"london, england",***,funding circle uk,5beea5e0-c2ab-4c5f-8c77-0a141a1c3bcc,"engineering manager funding circle uk london, ..."


In [49]:
all_music_jobs.all_info

0       primary music teacher - immediate start (qts e...
1       music teacher engage education brentwood, engl...
2           support worker consensus feltham, england ***
3       support worker/care assistant norwood new barn...
4       finance assistant (part-time) octave group cam...
5       music editor, apple music – uk and ireland app...
6       software engineer (functional programming) fun...
7       content coordinator hogarth worldwide london, ...
8       music content strategist, youtube youtube lond...
9       engineering manager funding circle uk london, ...
10      music content strategist, youtube the dots lon...
11      content manager (production music) bmg rights ...
12        studio assistant beauty pie london, england ***
13          studio assistant next plc london, england ***
14           uk head of music dice fm london, england ***
15      leisure and wellness coordinator porthaven car...
16         junior sales broker bionic london, england ***
17      junior

In [50]:
all_music_jobs.columns

Index(['job_title', 'location', 'job_description', 'company', 'job_uuid',
       'all_info'],
      dtype='object')

In [51]:
#initializing tfidf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.feature_extraction.text import CountVectorizer

tfidf_vectorizer = TfidfVectorizer(min_df=6, max_features=None, strip_accents='unicode',
        analyzer="word", token_pattern=r'\w{1,}', ngram_range=(1, 2),
        use_idf=1, smooth_idf=1, sublinear_tf=1)

tfidf_jobid = tfidf_vectorizer.fit_transform((all_music_jobs['all_info']))
tfidf_jobid

#https://buhrmann.github.io/sklearn-pipelines.html
#https://towardsdatascience.com/higher-accuracy-and-less-process-time-in-text-classification-with-lda-and-tf-idf-d2d949e344c3
#https://www.kaggle.com/kandij/job-analysis-content-recommendation

<6507x6713 sparse matrix of type '<class 'numpy.float64'>'
	with 225074 stored elements in Compressed Sparse Row format>

In [52]:
with open('uk_users.json', encoding='utf-8-sig') as f_input:
    uk_df = pd.read_json(f_input)  

uk_users = pd.DataFrame.from_dict(uk_df, orient='columns')
#https://stackoverflow.com/questions/41168558/python-how-to-convert-json-file-to-dataframe

In [53]:
uk_users.head()

,amazonMusic,appleMusic,bandcamp,biography,currentCompany,currentJob,deezer,email,facebook,headline,...,reasonsForJoining_offerOpportunities,reasonsForJoining_promoteServices,soundcloud,spotify,tiktok,twitter,uid,username,website,youtube
0,https://www.amazon.co.uk/sintmollitad/e/conseq...,https://music.apple.com/gb/artist/voluptate/10...,https://eiusmod.bandcamp.com/releases,Aute qui do et irure in. Laboris est laborum n...,Pholio,singer,https://deezer.com/en/artist/81069669,voluptateeu610@proidentadipisicingmollit.com,wwww.facebook.com/7886107710,Aliquip exercitation culpa nostrud dolor elit ...,...,False,False,https://www.soundcloud.com/1068101068,https://www.spotify.com/artist/98710767,@essecillum7899796,@fugiatofficia910678910,ae535943-f964-44da-9eda-853337abf040,Sophie Allison,https://culpaconsequatmagna.com,https://www.youtube.com/user/magnaex69106976}
1,https://www.amazon.co.uk/minimnonelit/e/auteir...,https://music.apple.com/gb/artist/esse/61077779,https://irure.bandcamp.com/releases,Tempor sint pariatur quis nulla in anim deseru...,Jimbies,talent partner,https://deezer.com/en/artist/9769968,nostrudfugiat87@laboreipsumet.com,wwww.facebook.com/810107610107,Ullamco nisi duis ipsum reprehenderit excepteu...,...,True,False,https://www.soundcloud.com/67610897,https://www.spotify.com/artist/1099106610,@consecteturincididunt7101076109,@irurereprehenderit610710678,e2e3ff83-5403-4131-bce2-b8c07a6afa2b,Jaime Gibson,https://mollitculpaamet.com,https://www.youtube.com/user/sintnostrud107761...
2,https://www.amazon.co.uk/deseruntidest/e/enimq...,https://music.apple.com/gb/artist/aliqua/6978788,https://anim.bandcamp.com/releases,Ad exercitation nisi amet incididunt. Excepteu...,Gology,music teacher,https://deezer.com/en/artist/86986106,aliquipsit86@deseruntmagnaad.com,wwww.facebook.com/998910668,Ex commodo voluptate consectetur do exercitati...,...,False,True,https://www.soundcloud.com/67971066,https://www.spotify.com/artist/6966966,@voluptatemollit910109688,@cillumsit71096789,4103b78c-e9fa-46d9-9263-943fb4935b44,Diane England,https://voluptatevelitaliqua.com,https://www.youtube.com/user/proidentnulla7981...
3,https://www.amazon.co.uk/cillumessepariatur/e/...,https://music.apple.com/gb/artist/minim/910967109,https://laboris.bandcamp.com/releases,Exercitation anim aute non deserunt velit eu l...,Comveyor,music producer,https://deezer.com/en/artist/779891010,esseeiusmod106@enimpariaturullamco.com,wwww.facebook.com/6710871079,Esse incididunt ipsum ex dolor occaecat esse s...,...,False,True,https://www.soundcloud.com/871097610,https://www.spotify.com/artist/8677886,@cupidatatdolore9869867,@adipisicingfugiat8610610107,a5fd4d58-0c14-455e-8045-4ad10155341c,Gayle Mckay,https://magnaetmollit.com,https://www.youtube.com/user/nullaex10786988}
4,https://www.amazon.co.uk/velitdeseruntirure/e/...,https://music.apple.com/gb/artist/fugiat/76981078,https://sit.bandcamp.com/releases,Cupidatat nulla magna ut culpa sint duis culpa...,Myopium,composer,https://deezer.com/en/artist/9899898,excepteuresse69@culpaexercitationsit.com,wwww.facebook.com/9109106877,Adipisicing enim nisi sit incididunt consectet...,...,True,False,https://www.soundcloud.com/8789998,https://www.spotify.com/artist/810786910,@dolorenisi79610977,@essenulla108661096,a81c575a-e863-444a-b3fe-a88fdac68ed9,Jennings Dominguez,https://excepteurexcepteursunt.com,https://www.youtube.com/user/eulabore1087107108}


In [54]:
isinstance(uk_users, pd.DataFrame)

True

In [55]:
uk_users.columns

Index(['amazonMusic', 'appleMusic', 'bandcamp', 'biography', 'currentCompany',
       'currentJob', 'deezer', 'email', 'facebook', 'headline', 'instagram',
       'interests', 'linkedin', 'location', 'musicalSkills', 'otherSkills',
       'pandora', 'phoneNumber', 'publicEmailAddress',
       'reasonsForJoining_connectOthers',
       'reasonsForJoining_findOpportunities',
       'reasonsForJoining_offerOpportunities',
       'reasonsForJoining_promoteServices', 'soundcloud', 'spotify', 'tiktok',
       'twitter', 'uid', 'username', 'website', 'youtube'],
      dtype='object')

In [56]:
with open('us_users.json', encoding='utf-8-sig') as f_input:
    us_df = pd.read_json(f_input)  

us_users = pd.DataFrame.from_dict(us_df, orient='columns')
#https://stackoverflow.com/questions/41168558/python-how-to-convert-json-file-to-dataframe

In [57]:
isinstance(us_users, pd.DataFrame)

True

In [58]:
all_users = pd.DataFrame()

all_users = all_users.append( [us_users, uk_users], ignore_index=True, verify_integrity=True, sort=False)

In [59]:
all_users.columns

Index(['amazonMusic', 'appleMusic', 'bandcamp', 'biography', 'currentCompany',
       'currentJob', 'deezer', 'email', 'facebook', 'headline', 'instagram',
       'interests', 'linkedin', 'location', 'musicalSkills', 'otherSkills',
       'pandora', 'phoneNumber', 'publicEmailAddress',
       'reasonsForJoining_connectOthers',
       'reasonsForJoining_findOpportunities',
       'reasonsForJoining_offerOpportunities',
       'reasonsForJoining_promoteServices', 'soundcloud', 'spotify', 'tiktok',
       'twitter', 'uid', 'username', 'website', 'youtube'],
      dtype='object')

In [60]:
all_users.head()

,amazonMusic,appleMusic,bandcamp,biography,currentCompany,currentJob,deezer,email,facebook,headline,...,reasonsForJoining_offerOpportunities,reasonsForJoining_promoteServices,soundcloud,spotify,tiktok,twitter,uid,username,website,youtube
0,https://www.amazon.co.uk/minimeaadipisicing/e/...,https://music.apple.com/gb/artist/elit/810109778,https://voluptate.bandcamp.com/releases,Qui commodo enim pariatur voluptate incididunt...,Scentric,PA,https://deezer.com/en/artist/71079786,cupidatatea107@officiaconsequatpariatur.com,wwww.facebook.com/910107109108,Et tempor occaecat commodo esse anim reprehend...,...,True,True,https://www.soundcloud.com/10108910108,https://www.spotify.com/artist/77108989,@consequatlaboris791097108,@consequatenim109667710,2e39953d-5733-4bd2-8f25-1ebc5ac3eb09,Sears Mcmahon,https://laborumeiusmoddo.com,https://www.youtube.com/user/magnaesse6866679}
1,https://www.amazon.co.uk/fugiatexmagna/e/conse...,https://music.apple.com/gb/artist/ex/91010101079,https://proident.bandcamp.com/releases,Aute amet dolore cupidatat minim nulla deserun...,Ginkle,events organiser,https://deezer.com/en/artist/8677996,exercitationconsequat77@ameteiusmodaute.com,wwww.facebook.com/8107810976,Velit fugiat veniam magna velit ipsum labore e...,...,True,False,https://www.soundcloud.com/678681010,https://www.spotify.com/artist/7977689,@excepteurquis86861078,@essecommodo7687689,48e1c60d-004f-4636-8f1c-fae608a4c36a,Eleanor Allen,https://deseruntconsecteturin.com,https://www.youtube.com/user/mollitirure99109866}
2,https://www.amazon.co.uk/enimproidentelit/e/al...,https://music.apple.com/gb/artist/consectetur/...,https://nostrud.bandcamp.com/releases,Reprehenderit eu nulla anim consectetur in lab...,Brainquil,session musician,https://deezer.com/en/artist/9897989,magnaamet76@consecteturdolorlaborum.com,wwww.facebook.com/79107981010,Culpa dolore veniam veniam sint nulla amet com...,...,True,False,https://www.soundcloud.com/101068779,https://www.spotify.com/artist/96106988,@exercitationqui691091076,@doloresunt96108766,459e6247-fe7d-47e9-8432-f1f5c3b5381a,Sallie Valencia,https://magnaofficiamagna.com,https://www.youtube.com/user/aliquipaliqua6810...
3,https://www.amazon.co.uk/inullamcoofficia/e/de...,https://music.apple.com/gb/artist/reprehenderi...,https://voluptate.bandcamp.com/releases,Adipisicing ullamco incididunt nisi voluptate ...,Verbus,music arranger,https://deezer.com/en/artist/91068789,fugiatullamco810@exenimnon.com,wwww.facebook.com/6910710667,Id nisi magna enim velit sit nisi enim ea inci...,...,False,True,https://www.soundcloud.com/791010897,https://www.spotify.com/artist/89689109,@estminim66610667,@aliquipet109881068,106798ea-95f3-4b5a-8d84-12f4df1d8313,Lilian Gallagher,https://voluptatenisiest.com,https://www.youtube.com/user/consecteturenim88...
4,https://www.amazon.co.uk/ullamcoesteiusmod/e/v...,https://music.apple.com/gb/artist/ut/71089697,https://deserunt.bandcamp.com/releases,Ad nostrud ea incididunt laboris esse eu eiusm...,Exovent,talent partner,https://deezer.com/en/artist/7108910610,magnaet87@quieacommodo.com,wwww.facebook.com/79688996,Ea et esse voluptate ex excepteur ipsum qui ex...,...,False,True,https://www.soundcloud.com/99810998,https://www.spotify.com/artist/89109787,@esseexercitation77109799,@quiminim766108107,7e35f8e4-fdb8-44aa-b96c-507a36ab5ab8,Harrell Camacho,https://estlaborumnon.com,https://www.youtube.com/user/laborummollit8991...


In [61]:
all_users.location.value_counts()

Edinburgh, United Kingdom                     55
Leeds, United Kingdom                         52
Belfast, United Kingdom                       48
Oxford, United Kingdom                        48
Brighton, United Kingdom                      48
Birmingham, United Kingdom                    47
Aberdeen, United Kingdom                      44
Bristol, United Kingdom                       43
Chester, United Kingdom                       43
London, United Kingdom                        42
Glasgow, United Kingdom                       42
Cardiff, United Kingdom                       41
Manchester, United Kingdom                    38
Sheffield, United Kingdom                     37
Coventry, United Kingdom                      37
Bath, United Kingdom                          35
Memphis, Tennessee, USA                       25
Tampa, Florida, USA                           22
Milwaukee, Wisconsin, USA                     21
San Jose, California, USA                     21
Las Vegas, Nevada, U

In [62]:
all_users.currentJob.value_counts()

music producer               60
dj                           59
sound engineer               44
music publicist              44
song-writer                  42
social media manager         41
lecturer                     38
tour manager                 38
drummer                      38
music marketing assistant    37
musical therapist            37
music teacher                36
stage manager                35
booking agent                35
concert booker               34
team leader                  34
bassist                      34
audio technician             34
content manager              33
singer                       32
audio designer               31
talent partner               30
music journalist             30
session musician             30
recording engineer           29
events organiser             29
music conductor              29
music agent                  28
guitarist                    28
client engagement manager    28
music photographer           28
music wr

In [63]:
all_users.shape

(1400, 31)

In [64]:
def user_processing(dataset):
    dataset = dataset.applymap(lambda s:s.lower() if type(s) == str else s)
    dataset = dataset.applymap(lambda s:s.replace('[^a-zA-Z \n\.]'," ") if type(s) == str else s)

    return dataset

user_processing(all_users)

,amazonMusic,appleMusic,bandcamp,biography,currentCompany,currentJob,deezer,email,facebook,headline,...,reasonsForJoining_offerOpportunities,reasonsForJoining_promoteServices,soundcloud,spotify,tiktok,twitter,uid,username,website,youtube
0,https://www.amazon.co.uk/minimeaadipisicing/e/...,https://music.apple.com/gb/artist/elit/810109778,https://voluptate.bandcamp.com/releases,qui commodo enim pariatur voluptate incididunt...,scentric,pa,https://deezer.com/en/artist/71079786,cupidatatea107@officiaconsequatpariatur.com,wwww.facebook.com/910107109108,et tempor occaecat commodo esse anim reprehend...,...,True,True,https://www.soundcloud.com/10108910108,https://www.spotify.com/artist/77108989,@consequatlaboris791097108,@consequatenim109667710,2e39953d-5733-4bd2-8f25-1ebc5ac3eb09,sears mcmahon,https://laborumeiusmoddo.com,https://www.youtube.com/user/magnaesse6866679}
1,https://www.amazon.co.uk/fugiatexmagna/e/conse...,https://music.apple.com/gb/artist/ex/91010101079,https://proident.bandcamp.com/releases,aute amet dolore cupidatat minim nulla deserun...,ginkle,events organiser,https://deezer.com/en/artist/8677996,exercitationconsequat77@ameteiusmodaute.com,wwww.facebook.com/8107810976,velit fugiat veniam magna velit ipsum labore e...,...,True,False,https://www.soundcloud.com/678681010,https://www.spotify.com/artist/7977689,@excepteurquis86861078,@essecommodo7687689,48e1c60d-004f-4636-8f1c-fae608a4c36a,eleanor allen,https://deseruntconsecteturin.com,https://www.youtube.com/user/mollitirure99109866}
2,https://www.amazon.co.uk/enimproidentelit/e/al...,https://music.apple.com/gb/artist/consectetur/...,https://nostrud.bandcamp.com/releases,reprehenderit eu nulla anim consectetur in lab...,brainquil,session musician,https://deezer.com/en/artist/9897989,magnaamet76@consecteturdolorlaborum.com,wwww.facebook.com/79107981010,culpa dolore veniam veniam sint nulla amet com...,...,True,False,https://www.soundcloud.com/101068779,https://www.spotify.com/artist/96106988,@exercitationqui691091076,@doloresunt96108766,459e6247-fe7d-47e9-8432-f1f5c3b5381a,sallie valencia,https://magnaofficiamagna.com,https://www.youtube.com/user/aliquipaliqua6810...
3,https://www.amazon.co.uk/inullamcoofficia/e/de...,https://music.apple.com/gb/artist/reprehenderi...,https://voluptate.bandcamp.com/releases,adipisicing ullamco incididunt nisi voluptate ...,verbus,music arranger,https://deezer.com/en/artist/91068789,fugiatullamco810@exenimnon.com,wwww.facebook.com/6910710667,id nisi magna enim velit sit nisi enim ea inci...,...,False,True,https://www.soundcloud.com/791010897,https://www.spotify.com/artist/89689109,@estminim66610667,@aliquipet109881068,106798ea-95f3-4b5a-8d84-12f4df1d8313,lilian gallagher,https://voluptatenisiest.com,https://www.youtube.com/user/consecteturenim88...
4,https://www.amazon.co.uk/ullamcoesteiusmod/e/v...,https://music.apple.com/gb/artist/ut/71089697,https://deserunt.bandcamp.com/releases,ad nostrud ea incididunt laboris esse eu eiusm...,exovent,talent partner,https://deezer.com/en/artist/7108910610,magnaet87@quieacommodo.com,wwww.facebook.com/79688996,ea et esse voluptate ex excepteur ipsum qui ex...,...,False,True,https://www.soundcloud.com/99810998,https://www.spotify.com/artist/89109787,@esseexercitation77109799,@quiminim766108107,7e35f8e4-fdb8-44aa-b96c-507a36ab5ab8,harrell camacho,https://estlaborumnon.com,https://www.youtube.com/user/laborummollit8991...
5,https://www.amazon.co.uk/culpamollitconsequat/...,https://music.apple.com/gb/artist/dolore/68881097,https://minim.bandcamp.com/releases,quis commodo duis duis cillum excepteur incidi...,anocha,social media manager,https://deezer.com/en/artist/66887108,loremut89@enimetsint.com,wwww.facebook.com/76101071066,officia cupidatat magna est nostrud.,...,False,False,https://www.soundcloud.com/9797996,https://www.spotify.com/artist/87881069,@autefugiat69797710,@adlabore9101076710,e2fde7eb-3c12-49dc-8b90-e4407ffbefa0,peters gonzalez,https://sintexincididunt.com,https://www.youtube.com/user/cillu

In [65]:
all_users['musicalSkills'] = all_users['musicalSkills'].apply(lambda x: ' '.join(map(str, x)))
all_users['otherSkills'] = all_users['otherSkills'].apply(lambda x: ' '.join(map(str, x)))
all_users['interests'] = all_users['interests'].apply(lambda x: ' '.join(map(str, x)))

#https://stackoverflow.com/questions/45306988/column-of-lists-convert-list-to-string-as-a-new-column

In [66]:
all_users["user_info"] = all_users["currentJob"].map(str) + " "+ all_users["musicalSkills"] + " " + all_users["otherSkills"] + " "

all_users['user_info'] = all_users['user_info'].str.replace('[^a-zA-Z \n\.]',"")

all_users['user_info'] = all_users['user_info'].str.lower()

all_users.head()

#https://www.kaggle.com/kandij/job-analysis-content-recommendation

,amazonMusic,appleMusic,bandcamp,biography,currentCompany,currentJob,deezer,email,facebook,headline,...,reasonsForJoining_promoteServices,soundcloud,spotify,tiktok,twitter,uid,username,website,youtube,user_info
0,https://www.amazon.co.uk/minimeaadipisicing/e/...,https://music.apple.com/gb/artist/elit/810109778,https://voluptate.bandcamp.com/releases,Qui commodo enim pariatur voluptate incididunt...,Scentric,PA,https://deezer.com/en/artist/71079786,cupidatatea107@officiaconsequatpariatur.com,wwww.facebook.com/910107109108,Et tempor occaecat commodo esse anim reprehend...,...,True,https://www.soundcloud.com/10108910108,https://www.spotify.com/artist/77108989,@consequatlaboris791097108,@consequatenim109667710,2e39953d-5733-4bd2-8f25-1ebc5ac3eb09,Sears Mcmahon,https://laborumeiusmoddo.com,https://www.youtube.com/user/magnaesse6866679},pa events management teaching music artist man...
1,https://www.amazon.co.uk/fugiatexmagna/e/conse...,https://music.apple.com/gb/artist/ex/91010101079,https://proident.bandcamp.com/releases,Aute amet dolore cupidatat minim nulla deserun...,Ginkle,events organiser,https://deezer.com/en/artist/8677996,exercitationconsequat77@ameteiusmodaute.com,wwww.facebook.com/8107810976,Velit fugiat veniam magna velit ipsum labore e...,...,False,https://www.soundcloud.com/678681010,https://www.spotify.com/artist/7977689,@excepteurquis86861078,@essecommodo7687689,48e1c60d-004f-4636-8f1c-fae608a4c36a,Eleanor Allen,https://deseruntconsecteturin.com,https://www.youtube.com/user/mollitirure99109866},events organiser singing music production drum...
2,https://www.amazon.co.uk/enimproidentelit/e/al...,https://music.apple.com/gb/artist/consectetur/...,https://nostrud.bandcamp.com/releases,Reprehenderit eu nulla anim consectetur in lab...,Brainquil,session musician,https://deezer.com/en/artist/9897989,magnaamet76@consecteturdolorlaborum.com,wwww.facebook.com/79107981010,Culpa dolore veniam veniam sint nulla amet com...,...,False,https://www.soundcloud.com/101068779,https://www.spotify.com/artist/96106988,@exercitationqui691091076,@doloresunt96108766,459e6247-fe7d-47e9-8432-f1f5c3b5381a,Sallie Valencia,https://magnaofficiamagna.com,https://www.youtube.com/user/aliquipaliqua6810...,session musician events management cello class...
3,https://www.amazon.co.uk/inullamcoofficia/e/de...,https://music.apple.com/gb/artist/reprehenderi...,https://voluptate.bandcamp.com/releases,Adipisicing ullamco incididunt nisi voluptate ...,Verbus,music arranger,https://deezer.com/en/artist/91068789,fugiatullamco810@exenimnon.com,wwww.facebook.com/6910710667,Id nisi magna enim velit sit nisi enim ea inci...,...,True,https://www.soundcloud.com/791010897,https://www.spotify.com/artist/89689109,@estminim66610667,@aliquipet109881068,106798ea-95f3-4b5a-8d84-12f4df1d8313,Lilian Gallagher,https://voluptatenisiest.com,https://www.youtube.com/user/consecteturenim88...,music arranger trumpet music production oboe r...
4,https://www.amazon.co.uk/ullamcoesteiusmod/e/v...,https://music.apple.com/gb/artist/ut/71089697,https://deserunt.bandcamp.com/releases,Ad nostrud ea incididunt laboris esse eu eiusm...,Exovent,talent partner,https://deezer.com/en/artist/7108910610,magnaet87@quieacommodo.com,wwww.facebook.com/79688996,Ea et esse voluptate ex excepteur ipsum qui ex...,...,True,https://www.soundcloud.com/99810998,https://www.spotify.com/artist/89109787,@esseexercitation77109799,@quiminim766108107,7e35f8e4-fdb8-44aa-b96c-507a36ab5ab8,Harrell Camacho,https://estlaborumnon.com,https://www.youtube.com/user/laborummollit8991...,talent partner djing singing artist management...


In [67]:
u = "459e6247-fe7d-47e9-8432-f1f5c3b5381a"
index = np.where(all_users['uid'] == u)[0][0]
user = all_users.iloc[[index]]
user
#https://www.kaggle.com/kandij/job-analysis-content-recommendation

,amazonMusic,appleMusic,bandcamp,biography,currentCompany,currentJob,deezer,email,facebook,headline,...,reasonsForJoining_promoteServices,soundcloud,spotify,tiktok,twitter,uid,username,website,youtube,user_info
2,https://www.amazon.co.uk/enimproidentelit/e/al...,https://music.apple.com/gb/artist/consectetur/...,https://nostrud.bandcamp.com/releases,Reprehenderit eu nulla anim consectetur in lab...,Brainquil,session musician,https://deezer.com/en/artist/9897989,magnaamet76@consecteturdolorlaborum.com,wwww.facebook.com/79107981010,Culpa dolore veniam veniam sint nulla amet com...,...,False,https://www.soundcloud.com/101068779,https://www.spotify.com/artist/96106988,@exercitationqui691091076,@doloresunt96108766,459e6247-fe7d-47e9-8432-f1f5c3b5381a,Sallie Valencia,https://magnaofficiamagna.com,https://www.youtube.com/user/aliquipaliqua6810...,session musician events management cello class...


In [68]:
from sklearn.metrics.pairwise import cosine_similarity
user_tfidf = tfidf_vectorizer.transform(user['user_info'])
output = map(lambda x: cosine_similarity(user_tfidf, x),tfidf_jobid)
output2 = list(output)

top = sorted(range(len(output2)), key=lambda i: output2[i], reverse=True)[:50]
recommendation = pd.DataFrame(columns = ['uuid', 'job_uuid'])
count = 0
for i in top:
    recommendation.set_value(count, 'uuid',u)
    recommendation.set_value(count,'job_uuid' ,all_music_jobs['job_uuid'][i])
    count += 1

#https://www.kaggle.com/kandij/job-analysis-content-recommendation

/Users/zara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.
/Users/zara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # This is added back by InteractiveShellApp.init_path()


In [69]:
recommendation

,uuid,job_uuid
0,459e6247-fe7d-47e9-8432-f1f5c3b5381a,3947e0d7-a52b-47b5-a927-5539201491b0
1,459e6247-fe7d-47e9-8432-f1f5c3b5381a,9b0db7e9-146e-4366-b2f4-259af8bafe2e
2,459e6247-fe7d-47e9-8432-f1f5c3b5381a,7444f912-9dfb-4c5c-8e6a-8d19eabaad85
3,459e6247-fe7d-47e9-8432-f1f5c3b5381a,ddcfd1ac-0137-4473-bf72-e39447053aeb
4,459e6247-fe7d-47e9-8432-f1f5c3b5381a,e4dd9e75-8e67-4587-93c5-296c4a4ab947
5,459e6247-fe7d-47e9-8432-f1f5c3b5381a,8c1b40dc-925f-4bea-814b-40d030c586a0
6,459e6247-fe7d-47e9-8432-f1f5c3b5381a,166fde43-70a0-468f-ba26-bdc95cdffed1
7,459e6247-fe7d-47e9-8432-f1f5c3b5381a,e0ad6b2f-9cce-41dc-b295-633d55c3b13a
8,459e6247-fe7d-47e9-8432-f1f5c3b5381a,ac0ba54d-cc73-4f64-a567-6631a2512b6e
9,459e6247-fe7d-47e9-8432-f1f5c3b5381a,24345138-5ad5-4962-95bf-991e5959b63c


In [70]:
nearestjobs = recommendation['job_uuid']
job_description = pd.DataFrame(columns = ['job_uuid','all_info'])
for i in nearestjobs:
    index = np.where(all_music_jobs['job_uuid'] == i)[0][0]    
    job_description.set_value(count, 'job_uuid',i)
    job_description.set_value(count, 'all_info', all_music_jobs['all_info'][index])
    count += 1
#https://www.kaggle.com/kandij/job-analysis-content-recommendation

/Users/zara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """
/Users/zara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [71]:
def print_recc():
    for index, row in job_description.iterrows():
        print(str(index) + job_description.loc[index, 'all_info'])
#https://www.kaggle.com/kandij/job-analysis-content-recommendation

In [72]:
all_users.loc[2,"user_info"] 

#https://www.kaggle.com/kandij/job-analysis-content-recommendation

'session musician events management cello classical guitar djing artist management clarinet ux design teamwork writing time management '

In [73]:
print_recc()

50 management assistant for established london artist management company (uk) london, united kingdom  everybody's management minimum of 5 years experience in artist management required
51assistant professor in classical guitar *** gainesville, fl ***
52 music & artist management assistant role (uk) london, united kingdom  anonymous minimum of 3 years’ experience in artist management preferred
53classical/modern guitar teacher *** guildford ***
54 management assistant (uk) giant artist management  london, united kingdom  giant artist management minimum 2 years experience within music industry required
55 assistant / coordinator (us) activist artists management  nashville, united states  activist artist management firm seeks full-time assistant / coordinator in nashville
56 senior artist manager (uk) artist management company  london, united kingdom  handle recruitment impeccable organisational and time management skills required
57resident musician (pianists and guitarists) *** worldwid

In [74]:
from sklearn import metrics


In [75]:
def provide_recc(user_uid):
    index = np.where(all_users['uid'] == user_uid)[0][0]
    user = all_users.iloc[[index]] 
    user_tfidf = tfidf_vectorizer.transform(user['user_info'])
    output = map(lambda x: cosine_similarity(user_tfidf, x),tfidf_jobid)
    output2 = list(output)
    print("*****")   
    print(all_users.loc[index,"user_info"])
    print("*****")
    top = sorted(range(len(output2)), key=lambda i: output2[i], reverse=True)[:50]
    recommendation_user = pd.DataFrame(columns = ['uuid', 'job_uuid'])
    count = 0
    for i in top:
        recommendation_user.set_value(count, 'uuid',u)
        recommendation_user.set_value(count,'job_uuid' ,all_music_jobs['job_uuid'][i])
        count += 1
    nearestjobs = recommendation_user['job_uuid']
    job_description_user = pd.DataFrame(columns = ['job_uuid','all_info'])
    
    for i in nearestjobs:
        index = np.where(all_music_jobs['job_uuid'] == i)[0][0]    
        job_description_user.set_value(count, 'job_uuid',i)
        job_description_user.set_value(count, 'all_info', all_music_jobs['all_info'][index])
        count += 1
        
    for index, row in job_description_user.iterrows():
        print(str(index) + job_description_user.loc[index, 'all_info'])
        
    

#https://www.kaggle.com/kandij/job-analysis-content-recommendation

In [76]:
text_to_format_1 = provide_recc("2e39953d-5733-4bd2-8f25-1ebc5ac3eb09")
text_to_format_1

*****
pa events management teaching music artist management djing oboe accordion video production ux design emotional intelligence teamwork 
*****


/Users/zara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/Users/zara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app
/Users/zara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/zara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


50orchestra librarian *** allentown, pa ***
51video production specialist *** oklahoma city, ok ***
52 client administrator (uk) entertainment intelligence  london, united kingdom  entertainment intelligence proven account management experience required
53 management assistant for established london artist management company (uk) london, united kingdom  everybody's management minimum of 5 years experience in artist management required
54 music & artist management assistant role (uk) london, united kingdom  anonymous minimum of 3 years’ experience in artist management preferred
55 management assistant (uk) giant artist management  london, united kingdom  giant artist management minimum 2 years experience within music industry required
56teaching artist, keyboard *** philadelphia, pa ***
57percussion teaching artist *** philadelphia, pa ***
58 executive and team assistant (uk) hardlivings artist management  london, united kingdom  hardlivings artist management this person must have a str

In [77]:
all_users.columns

Index(['amazonMusic', 'appleMusic', 'bandcamp', 'biography', 'currentCompany',
       'currentJob', 'deezer', 'email', 'facebook', 'headline', 'instagram',
       'interests', 'linkedin', 'location', 'musicalSkills', 'otherSkills',
       'pandora', 'phoneNumber', 'publicEmailAddress',
       'reasonsForJoining_connectOthers',
       'reasonsForJoining_findOpportunities',
       'reasonsForJoining_offerOpportunities',
       'reasonsForJoining_promoteServices', 'soundcloud', 'spotify', 'tiktok',
       'twitter', 'uid', 'username', 'website', 'youtube', 'user_info'],
      dtype='object')

In [78]:
all_users.loc[all_users['uid'] == "2e39953d-5733-4bd2-8f25-1ebc5ac3eb09"]
print

<function print>

In [79]:
text_to_format_2 = provide_recc("48e1c60d-004f-4636-8f1c-fae608a4c36a")
text_to_format_2

*****
events organiser singing music production drums harmonica electric guitar accordion teamwork video production problemsolving 
*****


/Users/zara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/Users/zara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app
/Users/zara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/zara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


50video production specialist *** oklahoma city, ok ***
51music production intern *** london administrative
52assistant professor of music production *** storm lake, ia ***
53game audio music production assistant intern *** san mateo, ca ***
54adjunct professor of music production *** york, pa ***
55production coordinator *** bethnal green, england ***
56p/t lab specialist, music production *** winter park, fl ***
57associate course director, music production *** winter park, fl ***
58music production intern *** santa monica, ca ***
59production assistant, uefa endeavor london, england ***
60 junior music manager (lx) jamendo  luxembourg  jamendo basic knowledge of music theory and music production needed
61associate teacher – singing & music *** london ***
62associate teacher – singing & music *** london ***
63career development music production industry outreach representative *** los angeles, ca ***
64video/audio production specialist: experienced *** new york, ny ***
65production a

Source 
https://www.kaggle.com/kandij/job-analysis-content-recommendation: used as a base for my algorithm